In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score
import random

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
def get_first_five_words(sentence):
    words = sentence.split()  # Split the sentence into a list of words
    return " ".join(words[:10])  # Join the first 5 words back into a string

In [4]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_grocery/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)
for _, row in df_recommender_train.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [5]:
df_recommender_train

,user_id,user_previous_interactions,recommended_product,negative_recommended_product,turns
0,AHG34BLPBFJAZLKSZTRSVPBYMNQA,"[{'product_name': 'Ateco Food Coloring Kit, 6 ...",{'product_name': 'ateco 1112 12-color food col...,[{'product_name': 'McCormick Nature's Inspirat...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,AFOSCJK6ARZVTE42E2BOYOCVSPWQ,[],"{'product_name': 'welpac panko bread crumbs', ...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,AGWDYYVVWM3DC3CASUZKXK67G6IA,"[{'product_name': 'Tazo 149901 Tea Bags, Calm ...","{'product_name': 'sucanat sugar, 1 lb.', 'pare...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,AFNVEHQGMJHJFM4I7ZJPRXCZFMZA,[],"{'product_name': 'senseo coffee pods, kona ble...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,AFG357TH7OYHATTMDUYIVEE2M4UQ,[],{'product_name': 'better than bouillon premium...,[{'product_name': 'Herb-Ox Sodium Free Bouillo...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
8401,AGQ7M4IO7E2S2GXHQY2SNFG2ATOA,"[{'product_name': 'Bar Harbor All Sauce, White...",{'product_name': 'american gumball company ass...,"[{'product_name': 'Large 1"" Blue Gumballs - 2 ...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8402,AH7PP6TAJYHFIMUC4WLFCG44AJSQ,"[{'product_name': 'STONE COLD JO: 12 oz, Cold ...",{'product_name': 'breville nespresso vertuo co...,[{'product_name': 'Dark Chocolate Covered Roas...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8403,AGEAKAPK6PVQHRBE5XNWTJPRQCKQ,[{'product_name': 'Sunbest Natural - Jumbo Roa...,{'product_name': 'iya's all natural cassava fl...,[{'product_name': 'Terrasoul Superfoods Organi...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8404,AHCWAEALGQUJSPXMEZIB3EADS2RA,[{'product_name': 'Dress My Cupcake Decorating...,{'product_name': 'tate's bake shop white choco...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [6]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_grocery/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)
for _, row in df_recommender_validation.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [7]:
df_recommender_validation

,user_id,user_previous_interactions,recommended_product,negative_recommended_product,turns
0,AHKGN55E2F6I3H4HIWJNBAX2VZ2Q,[{'product_name': 'Chat Masala 2.0oz By Zamour...,{'product_name': 'minor's lobster base - 1 lb....,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,AGRW7YJ4PIFMVLZY6WQ6BH2XXAYA,[{'product_name': 'Zint Organic Cacao Powder (...,{'product_name': 'eden organic traditional bar...,"[{'product_name': 'Barley Malt Powder, 1 lb.',...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,AHVECP5Y4WSYC3T26A3QHRGYLNDQ,[{'product_name': 'Pompeian USDA Organic Robus...,"{'product_name': 'now foods, certified organic...",[{'product_name': 'Brazil Nuts Roasted Salted ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,AEQ5A4PCOLD3DEWCE5HHFDF7IGSA,[{'product_name': 'ORGAIN Organic Strawberries...,{'product_name': 'anchor full cream milk powde...,"[{'product_name': 'Nestle Nido Milk Powder, Im...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,AG23NKQFFCJYUY6IAZWHJA4GLFXA,"[{'product_name': '365 by Whole Foods Market, ...",{'product_name': 'daechun(choi's1) seaweed sna...,[{'product_name': 'Annie Chun's - Crispy Organ...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
1796,AGMZTOBO3IELI54QTURQADHVFOJQ,[{'product_name': 'Organic Instant Coffee Powd...,{'product_name': 'angie's boomchickapop milk c...,"[{'product_name': 'ORGANIC, 1.9 LB Heirloom Mu...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1797,AEAXAJACFMXIAAH4WOHRMXPSZWFA,"[{'product_name': 'Timothy's World Coffee, Kon...",{'product_name': 'good nature organic chamomil...,"[{'product_name': 'VAHDAM, Chai Tea Variety Sa...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1798,AEHQILYLFLHZ2OONHOQCIHHYL2QA,[{'product_name': 'Prima Taste Rendang Curry S...,{'product_name': 'lao gan ma (laoganma) spicy ...,"[{'product_name': 'Campbell's Chunky Soup, Hot...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1799,AGVFUTAWFKJFHNEPRNMGENY7SYRA,[{'product_name': 'Brewer's Best Beer Ingredie...,{'product_name': 'perfectware sweet corn glaze...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [8]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
prompt_train = []
recommend_train = []
not_founds = 0

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = ""
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
        
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_train.append(prompt)
                recommend_train.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"len(prompt_train): {len(prompt_train)}")
print(f"len(recommend_train): {len(recommend_train)}")

            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

  0%|          | 0/8406 [00:00<?, ?it/s]

100%|██████████| 8406/8406 [01:40<00:00, 83.32it/s] 


Could not find 113
len(prompt_train): 8293
len(recommend_train): 8293


In [10]:
prompt_validation = []
recommend_validation = []
not_founds = 0

for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = ""
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
            
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_validation.append(prompt)
                recommend_validation.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1
        
print(f"Could not find {not_founds}")
print(f"len(prompt_validation): {len(prompt_validation)}")
print(f"len(recommend_validation): {len(recommend_validation)}")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

  0%|          | 0/1801 [00:00<?, ?it/s]

100%|██████████| 1801/1801 [00:21<00:00, 82.78it/s] 


Could not find 30
len(prompt_validation): 1771
len(recommend_validation): 1771


In [11]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [12]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/amazon_grocery/T5_recommender",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
trainer.train() # resume_from_checkpoint=True
trainer.save_model()

Step,Training Loss,Validation Loss
1037,2.526800,2.405329
2074,2.269900,2.301026
3111,2.173600,2.277305


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [14]:
torch.cuda.empty_cache()

In [15]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_grocery/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)
for _, row in df_recommender_test.iterrows():
    row["recommended_product"]["product_name"] = row["recommended_product"]["product_name"].lower()

In [16]:
apps_training_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/amazon_grocery/grocery_df.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(get_first_five_words(row["title"].lower()))
        
all_apps = list(set(all_apps))

In [17]:
def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d( all_apps, [get_first_five_words(row["recommended_product"]["product_name"])]), 24, replace=False) # filter_candidate_apps(row["recommended_book"]["book_name"])
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, get_first_five_words(row["recommended_product"]["product_name"])) 

df_recommender_test['candidate'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [18]:
prompt_test = []
recommend_test = []
candidate_books = []
true_candidate_indexes = []
not_founds = 0
for _, row in df_recommender_test.iterrows():
    candidates = []
    for index, candidate_book in enumerate(row["candidate"].tolist()):
        candidates.append(candidate_book)
        if candidate_book == get_first_five_words(row["recommended_product"]["product_name"]):
            true_candidate_index = index
    prompt = ""
    
    found = False
    recommended = get_first_five_words(row["recommended_product"]["product_name"])
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(recommended)
            candidate_books.append(candidates)
            true_candidate_indexes.append(true_candidate_index)
            found = True
            break
        else:
            prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"Number of prompt: {len(prompt_test)}")
print(f"Number of generations: {len(recommend_test)}")
print(f"Number of candidate apps: {len(candidate_books)}")
print(f"Number of true candidate indexes: {len(true_candidate_indexes)}")

Could not find 18
Number of prompt: 1784
Number of generations: 1784
Number of candidate apps: 1784
Number of true candidate indexes: 1784


In [19]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/amazon_grocery/T5_recommender")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True) for generation in generations_predicted]
    generation_batch = [generation for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [21]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

  0%|          | 0/223 [00:00<?, ?it/s]

100%|██████████| 223/223 [04:32<00:00,  1.22s/it]

success_rate:  0.12836322869955158


In [ ]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
                
  return scores

In [23]:
scores = recommender_rank(prompt_test, candidate_books, model, tokenizer, batch_size=8)

100%|██████████| 5575/5575 [20:55<00:00,  4.44it/s]


Sampled Candidates

In [24]:
# that is the one
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

[np.float64(0.6266816143497758),
 np.float64(0.7320627802690582),
 np.float64(0.7797085201793722),
 np.float64(0.820627802690583),
 np.float64(0.8492152466367713),
 np.float64(0.8710762331838565),
 np.float64(0.890695067264574),
 np.float64(0.9019058295964125),
 np.float64(0.9131165919282511),
 np.float64(0.9209641255605381)]

In [25]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [26]:
# that is the one
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[np.float64(0.6266816143497758),
 np.float64(0.6931697273943016),
 np.float64(0.7169925973494585),
 np.float64(0.7346155731002196),
 np.float64(0.7456747060424626),
 np.float64(0.7534617465678063),
 np.float64(0.7600013579280454),
 np.float64(0.7635379596857329),
 np.float64(0.7669127354218762),
 np.float64(0.7691811813683169)]

Similar Candidates

In [ ]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

In [44]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [ ]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]